# アウトバウンド認証

アウトバウンド認証により、エージェントと AgentCore Gateway は、インバウンド認証中に認証・認可されたユーザーに代わって、AWS リソースやサードパーティサービスに安全にアクセスできます。AWS リソースやサードパーティサービスとの認可を統合するには、インバウンド認証とアウトバウンド認証の両方を構成する必要があります。

AgentCore Identity がサポートする必要最小限のアクセス権限と安全な権限委譲により、エージェントは GitHub、Google、Salesforce、Slack などの AWS リソースやサードパーティツールにシームレスかつ安全にアクセスできます。エージェントは、事前に承認されたユーザーの同意がある場合、ユーザーに代わって、または独立してこれらのサービスでアクションを実行できます。さらに、安全なトークンボールトを使用して同意の負担を軽減し、合理化された AI エージェント体験を作成できます。

## アウトバウンド認証の構成

まず、サードパーティプロバイダーにクライアントアプリケーションを登録し、次にアウトバウンド認証を作成します。AWS リソース、サードパーティサービス、または AgentCore Gateway ターゲットへのアクセスを検証する方法を指定します。OAuth 2LO/3LO または API キーを使用できます。OAuth では、AgentCore Identity が提供するプロバイダーから選択できます。その場合、AgentCore Identity からプロバイダーの構成詳細を入力します。または、カスタムプロバイダーの詳細を指定することもできます。

ユーザーが AWS リソース、サードパーティサービス、または AgentCore Gateway ターゲットへのアクセスを希望すると、アウトバウンド認証はインバウンド認証によって提供されたアクセストークンが有効であることを確認し、有効であればリソースへのアクセスを許可します。

<div style="text-align:center">
    <img src="images/outbound_auth.png" width="90%"/>
</div>

## リソース資格情報プロバイダー

これは、エージェントが下流のリソースサーバー（例：Google、GitHub）の資格情報を取得してアクセスするために使用するコンポーネントです（例：GitHub からプライベートリポジトリを取得）。エンドユーザー、エージェントコード、外部認可サーバー間で 2LO および 3LO OAuth2 オーケストレーションフローを実装するエージェント開発者の負担を軽減します。AgentCore は、カスタム OAuth2 資格情報プロバイダーと、Google、GitHub、Slack、Salesforce などの組み込みプロバイダーのリストを提供しており、認可サーバーエンドポイントとプロバイダー固有のパラメータが事前に入力されています。

Bedrock AgentCore Identity は、OAuth2 または API キーをサポートする外部リソースで認証するためのエージェント開発者向けに OAuth2 および API キー資格情報プロバイダーを提供します。以下の例では、API キー資格情報プロバイダーの構成方法を説明します。エージェントは、API キー資格情報プロバイダーを使用して、エージェント操作用の API キーを取得できます。他の資格情報プロバイダーについては、ドキュメントを参照してください。

以下は、@require_access_token デコレータで使用できる各種パラメータです。

| パラメータ名          | 説明                                                                     |
|:--------------------|:-------------------------------------------------------------------------|
| provider_name       | 資格情報プロバイダー名                                                     |
| into                | トークンを注入するパラメータ名                                              |
| scopes              | リクエストする OAuth2 スコープ                                              |
| on_auth_url	      | 認可 URL を処理するコールバック                                            |
| auth_flow           | 認証フロータイプ（"M2M" または "USER_FEDERATION"）                         |
| callback_url        | OAuth2 コールバック URL                                                   |
| force_authentication| 再認証を強制                                                              |
| token_poller        | カスタムトークンポーラー実装                                               |

# Amazon Bedrock AgentCore Runtime での Strands エージェントのホスティング

## 概要

このチュートリアルでは、ユーザーの GitHub リポジトリからプライベートリポジトリを一覧表示できる Strands エージェントを使用してエージェントを開発します。GitHub との資格情報管理を支援する資格情報プロバイダーを構成します。GitHub の名前付きプロバイダーを使用し、エージェントコードを変更して資格情報プロバイダーを呼び出し、access_token を使用して GitHub からプライベートリポジトリのリストを取得できます。

### チュートリアルアーキテクチャ

<div style="text-align:center">
    <img src="images/outbound_auth_3lo.png" width="90%"/>
</div>


### チュートリアルの詳細

| 情報                | 詳細                                                                     |
|:--------------------|:-------------------------------------------------------------------------|
| チュートリアルタイプ  | 会話型                                                                   |
| エージェントタイプ   | シングル                                                                  |
| Agentic フレームワーク | Strands Agents                                                         |
| LLM モデル          | Anthropic Claude Haiku 4.5                                              |
| チュートリアルコンポーネント | AgentCore Runtime でのエージェントホスティング。Strands Agent と Claude Model を使用 |
| チュートリアル業種   | 業種横断                                                                  |
| 難易度              | 中級                                                                      |
| 使用 SDK            | Amazon BedrockAgentCore Python SDK と boto3                              |
| 資格情報プロバイダー | タイプ：OAuth2 - Github プロバイダー                                       |


### チュートリアルの主な機能

* Amazon Bedrock AgentCore Runtime でのエージェントホスティング
* Claude モデルの使用
* Strands Agents の使用
* OAuth2 Github 資格情報プロバイダーを使用した AgentCore エグレス認証の使用

## 前提条件

このチュートリアルを実行するには、以下が必要です：
* Python 3.10+
* AWS 資格情報
* Amazon Bedrock AgentCore SDK
* Strands Agents
* Docker が実行中であること

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

## IDP として Cognito を使用したインバウンド認証の構成

アプリクライアントと 1 人のテストユーザーを持つ Cognito ユーザープールをプロビジョニングしましょう。デプロイされた MCP サーバーにアクセスするための JWT トークンを提供するために Amazon Cognito を使用します。これを行うために、`utils` スクリプトの `setup_cognito_user_pool` サポート関数を使用します。

注意：Cognito access_token は 2 時間のみ有効です。access_token の有効期限が切れた場合は、`reauthenticate_user` メソッドを使用して別の access_token を取得できます。

In [ ]:
import sys
import os
from boto3.session import Session


# Get the current notebook's directory
current_dir = os.path.dirname(os.path.abspath('__file__' if '__file__' in globals() else '.'))

utils_dir = os.path.join(current_dir, '..')
utils_dir = os.path.abspath(utils_dir)

# Add to sys.path
sys.path.insert(0, utils_dir)
print("sys.path[0]:", sys.path[0])
from utils import setup_cognito_user_pool, reauthenticate_user

boto_session = Session()
region = boto_session.region_name
print(f"Region: {region}")

In [ ]:
print("Setting up Amazon Cognito user pool...")
cognito_config = setup_cognito_user_pool("Cognito_3LO_Github")
print("Cognito setup completed ✓")

## OAuth2 用の GitHub の構成（ユーザー代理フロー）

## GitHub で OAuth クライアントを作成する手順（client_id と client_secret の取得）

以下の手順に従って GitHub で OAuth アプリを作成し、他のアプリケーションで使用するために必要な資格情報を取得します：

1. **GitHub にサインイン**
   - [github.com](https://github.com) にアクセスし、アカウントにログインします。

2. **開発者設定にアクセス**
   - 右上隅のプロフィール写真をクリックします。
   - ドロップダウンから **Settings** を選択します。
   - 左サイドバーで下にスクロールし、**Developer settings** をクリックします。

3. **OAuth Apps に移動**
   - Developer settings の左サイドバーで **OAuth Apps** をクリックします。

4. **新しい OAuth アプリケーションを登録**
   - **New OAuth App**（または以前に作成したことがない場合は **Register a new application**）をクリックします。
   - **必要な詳細を入力：**
     - **Application Name**：アプリに名前を付けます。
     - **Homepage URL**：アプリのホームページの URL。<br>
        例：https://github.com/awslabs/amazon-bedrock-agentcore-samples/examplehomepage
     - **Application Description**（オプション）：わかりやすいように説明を追加します。
     - **Authorization callback URL**：認可後にユーザーが送信される URL（アプリケーションの OAuth フローで使用）。<br>
        例：https://bedrock-agentcore.us-east-1.amazonaws.com/identities/oauth2/callback/tobeupdated <br>
      - 注意：これは「Github OAuth2 資格情報プロバイダーの構成」ステップで実際のコールバック URL を受け取った後に更新する必要があります

     - GitHub 構成を示すスクリーンキャプチャはこちらです。
     <div style="text-align:left">
      <img src="images/github_details.png" width="80%"/>
      </div>
   - 上の画像に示すように、**Enable Device Flow** のチェックは外したままにします。
   - 詳細を入力したら、**Register application** をクリックします。

5. **資格情報の取得**
   - アプリケーションが登録されると、アプリケーションサマリーページに **Client ID** が表示されます。
   - **Client Secret** を生成するには、利用可能なボタンまたはリンクをクリックします。シークレットが表示されます—安全にコピーして保存してください。アプリケーションで使用します。
     - 注意：クライアントシークレットはセキュリティのため、1 回または数回のみ表示されます。

***

### 資格情報の使用

- Github 資格情報プロバイダーを構成する際に **client_id** と **client_secret** を使用します。
- クライアントシークレットを公開したりユーザーと共有したりしないでください—パスワードのように扱います。

***

## OAuth2 認可 URL セッションバインディングプロセス

OAuth2 認可 URL セッションバインディングプロセスは、OAuth2 認可セッションが AgentCore Identity で認証されたユーザーと適切に関連付けられることを保証する重要なセキュリティメカニズムです。このプロセスはセッションハイジャックを防ぎ、OAuth トークンが意図されたユーザーにのみ付与されることを保証します。

参照：https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/oauth2-authorization-url-session-binding.html

### セッションバインディングの仕組み
<div style="text-align:center">
    <img src="images/identity-session-binding.png" width="90%"/>
</div>

1. エージェントの呼び出し – エージェントユーザーが所有するアプリケーションやリソースにアクセスしたい場合、エージェントコードは GetResourceOauth2Token API を呼び出して認可 URL を取得します。

2. 認可 URL の生成 – AgentCore Identity はユーザーがナビゲートしてアクセスに同意するための認可 URL とセッション URI を生成します。

3. 認可とアクセストークンの取得 – ユーザーは認可 URL にナビゲートし、エージェントがリソースにアクセスすることに同意します。その後、AgentCore Identity はユーザーのブラウザを、認可リクエストの発信ユーザーに関する情報を含む HTTPS アプリケーションエンドポイントにリダイレクトします。この時点で、HTTPS アプリケーションエンドポイントは、発信エージェントユーザーがアプリケーションの現在ログインしているユーザーと同じかどうかを判断します。一致する場合、アプリケーションエンドポイントは CompleteResourceTokenAuth を呼び出し、AgentCore Identity がアクセストークンをフェッチして保存できるようにします。

4. アクセストークンを取得するためのエージェントの再呼び出し – アプリケーションが有効な応答を返すと、エージェントアプリケーションはユーザーのために最初にリクエストされた OAuth2.0 アクセストークンを取得できるようになります。ユーザーが一致しない場合、アプリケーションは何もしないか、試行をログに記録します。

アプリケーションエンドポイントがユーザー ID を検証できるようにすることで、AgentCore Identity はエージェントアプリケーションが認可リクエストを開始したユーザーとアクセスに同意したユーザーが常に同じであることを保証できます。

### このサンプルでの OAuth2 セッションバインディングフローの概要

OAuth2 セッションバインディングプロセスには、アプリケーション、AgentCore Identity、外部 OAuth プロバイダー（Github、Google など）、ローカルコールバックサーバー間で調整するいくつかの重要なステップが含まれます：

#### ステップ 1：アプリケーションコールバック URL の作成
- アプリケーションでパブリックにアクセス可能な HTTPS コールバックエンドポイントを作成
- このエンドポイントは OAuth リダイレクトを処理し、ユーザーセッションを検証
- 例：`https://myagentapp.com/callback`

#### ステップ 2：コールバック URL でワークロード ID を更新
- ワークロード ID にコールバック URL を `AllowedResourceOauth2ReturnUrl` として登録
- これは `UpdateWorkloadIdentity` API を使用して行います
- **このチュートリアルでは**：このステップは、環境対応のコールバックサーバー URL でワークロード ID を更新する以下のコードによって自動的に処理されます

#### ステップ 3：OAuth2 資格情報プロバイダーの作成
- 外部 OAuth プロバイダーの詳細（クライアント ID、シークレット）で資格情報プロバイダーを構成
- AgentCore Identity はプロバイダー用の一意のコールバック URL を返します
- このコールバック URL を外部 OAuth プロバイダー（例：GitHub Console）に登録

#### ステップ 4：セッション検証とトークン完了の実装
- コールバックエンドポイントは現在のユーザーのセッションを検証する必要があります
- ユーザー識別子とセッション URI で `CompleteResourceTokenAuth` API を呼び出す
- **このチュートリアルでは**：`oauth2_callback_server.py` がこれを自動的に処理します

#### ステップ 5：OAuth フローの実行
- ユーザーがエージェントとのインタラクションを通じて OAuth フローをトリガー
- ユーザーは認可のために外部プロバイダーにリダイレクト
- プロバイダーはセッション情報と共にコールバックにリダイレクト
- セッションバインディングが完了し、OAuth トークンが利用可能に

### 環境対応 OAuth2 コールバックサーバー

このチュートリアルでは、さまざまな実行環境に自動的に適応する環境対応の `oauth2_callback_server.py` を使用します：

#### **ローカル開発：**
- **外部コールバック URL**：`http://localhost:9090/oauth2/callback`（ブラウザからアクセス可能）
- **内部通信**：`http://localhost:9090`（ノートブック ↔ サーバー）
- **サーバーバインディング**：`127.0.0.1`（localhost のみ、安全）

#### **SageMaker Workshop Studio：**
- **外部コールバック URL**：`https://<domain>.studio.<region>.sagemaker.aws/proxy/9090/oauth2/callback`（プロキシ経由でブラウザからアクセス可能）
- **内部通信**：`http://localhost:9090`（同じコンテナ内のノートブック ↔ サーバー）
- **サーバーバインディング**：`0.0.0.0`（SageMaker プロキシがサーバーに到達可能）

OAuth2 コールバックサーバーは `/opt/ml/metadata/resource-metadata.json` をチェックして環境を自動検出し、それに応じて構成します。

### oauth2_callback_server.py の機能

1. **ローカル FastAPI サーバーを実行**（ポート 9090）
   - ヘルスチェック用の `/ping` エンドポイントを提供
   - ユーザートークンを保存する `/userIdentifier/token` エンドポイントを提供
   - OAuth リダイレクトを処理する `/oauth2/callback` エンドポイントを提供

2. **ユーザートークンストレージの管理**
   - Cognito 認証からのユーザーの JWT トークンを保存
   - OAuth セッションを正しいユーザー ID に関連付け

3. **OAuth コールバック処理の実行**
   - `session_id` パラメータ付きの OAuth リダイレクトを受信
   - セッションをバインドするために `CompleteResourceTokenAuth` を呼び出す
   - フローを完了する前にユーザー ID を検証

4. **セッションセキュリティの提供**
   - OAuth セッションが認証済みユーザーにバインドされることを保証
   - OAuth トークンへの不正アクセスを防止

5. **環境検出**
   - ローカル環境と SageMaker Studio 環境を自動検出
   - URL とサーバーバインディングを適切に構成

### ワークロード ID 更新との統合

コードは OAuth2 セッションバインディングプロセスの重要なステップを実行します：

```python
if launch_result.agent_id:
    workload_name = launch_result.agent_id
    workload_identity = identity_client.get_workload_identity(name=workload_name)
    allowed_resource_oauth_2_return_urls = workload_identity.get("allowedResourceOauth2ReturnUrls") or []
    oauth2_callback_url = get_oauth2_callback_url()
    print(f"Updating workload {workload_name} with callback url {oauth2_callback_url}")

    updated_workload_identity = identity_client.update_workload_identity(
        name=workload_name,
        allowed_resource_oauth_2_return_urls=[*allowed_resource_oauth_2_return_urls, oauth2_callback_url],
    )
```

このコードは：
1. **エージェントのワークロード ID を取得**：ランタイムデプロイメントからエージェント ID を使用
2. **現在の許可 URL を取得**：既存の `allowedResourceOauth2ReturnUrls` をフェッチ
3. **環境対応コールバック URL を追加**：
   - ローカル：`http://localhost:9090/oauth2/callback`
   - SageMaker：`https://<domain>.studio.sagemaker.aws/proxy/9090/oauth2/callback`
4. **ワークロード ID を更新**：AgentCore Identity にコールバック URL を登録

AgentCore Identity は事前登録された URL にのみ OAuth コールバックをリダイレクトするため、この登録は不可欠であり、追加のセキュリティ層を提供します。

### セキュリティに関する考慮事項

OAuth2 セッションバインディングプロセスには、いくつかのセキュリティ対策が含まれています：
- **URL 検証**：事前登録されたコールバック URL のみが許可
- **セッション検証**：トークン完了前にユーザーセッションを検証する必要がある
- **ユーザー ID バインディング**：OAuth セッションは認証済みユーザーに明示的にバインド
- **トークン分離**：各ユーザーの OAuth トークンは分離され安全
- **環境対応 URL**：各環境に適切な URL を自動的に使用

この包括的なアプローチにより、OAuth2 フローがマルチユーザー環境で安全であり、ローカルで実行しても SageMaker Workshop Studio で実行しても正しいユーザーに適切に帰属することが保証されます。

---

#### Github OAuth2 資格情報プロバイダーの構成

以下のコードを変更し、以下を置き換えてください：
1. <your-github-client-id> を上記ステップ 5 で記録した「client id」に
2. <your-github-client-secret> を上記ステップ 5 で記録した「client secret」に

重要：「<」と「>」を含むプレースホルダー文字列全体を置き換えてください


クライアント ID とクライアントシークレットを更新したら、以下のコードを実行して Github の資格情報プロバイダーを作成します。<br>
AgentCore Identity のリソース資格情報プロバイダーは、エージェント、ID プロバイダー、リソースサーバー間の複雑な関係を管理するインテリジェントな仲介者として機能します。各プロバイダーは、特定のサービスまたは ID システムに必要な固有のエンドポイント構成をカプセル化します。このサービスは、Google、GitHub、Slack、Salesforce などの人気のあるサービス向けの組み込みプロバイダーを提供しており、認可サーバーエンドポイントとプロバイダー固有のパラメータが事前構成されているため、開発作業を削減できます。AgentCore Identity は、OAuth2 互換のリソースサーバーで動作するように調整できる構成可能な OAuth2 資格情報プロバイダーを通じてカスタム構成をサポートします。

In [ ]:
from bedrock_agentcore.services.identity import IdentityClient
region = boto_session.region_name
identity_client = IdentityClient(region)

# Configure GitHub OAuth2 provider - On-Behalf-Of User
github_provider = identity_client.create_oauth2_credential_provider({
    "name": "github-provider",
    "credentialProviderVendor": "GithubOauth2",
    "oauth2ProviderConfigInput": {
        "githubOauth2ProviderConfig": {
            'clientId': "<your-github-client-id>",
            'clientSecret': "<your-github-client-secret>"
        }
    }
})
print(github_provider)
print("\n")
print(f"callbackUrl: {github_provider['callbackUrl']}")

## GitHub で Authorization callback URL を更新する手順

1. **GitHub にサインイン**
   - [github.com](https://github.com) にアクセスし、アカウントにログインします。

2. **開発者設定にアクセス**
   - 右上隅のプロフィール写真をクリックします。
   - ドロップダウンから **Settings** を選択します。
   - 左サイドバーで下にスクロールし、**Developer settings** をクリックします。

3. **OAuth Apps に移動**
   - Developer settings の左サイドバーで **OAuth Apps** をクリックします。

4. **OAuth App をクリック**
   - 更新する必要がある、以前のステップで作成した OAuth App を見つけます。
   - OAuth App の名前をクリックして設定を開きます。

5. **Authorization callback URL を更新**
   - **Authorization callback URL** までスクロールダウンします
   - 上記の callbackUrl: URL をテキストボックスに入力し、**Update application** をクリックします。

In [ ]:
# Get the OAuth2 callback URL based on the current environment (notebook/SageMaker)
# This is evaluated HERE in the notebook, not in the agent container
from oauth2_callback_server import get_oauth2_callback_url
oauth2_callback_url_for_agent = get_oauth2_callback_url()

print(f"Callback URL for agent (determined from notebook environment): {oauth2_callback_url_for_agent}")

# Write github_agent.py with the callback URL embedded as a string literal
github_agent_code = f'''
import asyncio
import json
import os
from typing import Optional

import httpx
from bedrock_agentcore import BedrockAgentCoreApp
from bedrock_agentcore.identity.auth import requires_access_token
from strands import Agent, tool

# Environment configuration
os.environ["STRANDS_OTEL_ENABLE_CONSOLE_EXPORT"] = "true"
os.environ["OTEL_PYTHON_EXCLUDED_URLS"] = "/ping,/invocations"

# Global token storage (could be improved with a proper state management solution)
github_access_token: Optional[str] = None

app = BedrockAgentCoreApp()


@tool
def inspect_github_repos() -> str:
    """Inspect and list the user's private GitHub repositories.

    Returns:
        str: A JSON string containing the list of repositories and their details,
            or an authentication required message.
    """
    global github_access_token

    if not github_access_token:
        return json.dumps({{
            "auth_required": True,
            "message": "GitHub authentication is required. Please wait while we set up the authorization.",
            "events": []
        }})

    print(f"Using GitHub access token: {{github_access_token[:10]}}...")

    headers = {{"Authorization": f"Bearer {{github_access_token}}"}}

    try:
        with httpx.Client() as client:
            # Get user information
            user_response = client.get("https://api.github.com/user", headers=headers)
            user_response.raise_for_status()
            username = user_response.json().get("login", "Unknown")
            print(f"✅ User: {{username}}")

            # Search for user's repositories
            repos_response = client.get(
                f"https://api.github.com/search/repositories?q=user:{{username}}",
                headers=headers
            )
            repos_response.raise_for_status()
            repos_data = repos_response.json()
            print(f"✅ Found {{len(repos_data.get('items', []))}} repositories")

            repos = repos_data.get('items', [])
            if not repos:
                return f"No repositories found for {{username}}."

            # Format repository information
            response_lines = [f"GitHub repositories for {{username}}:\\n"]

            for repo in repos:
                repo_line = f"📁 {{repo['name']}}"
                if repo.get('language'):
                    repo_line += f" ({{repo['language']}})"
                repo_line += f" - ⭐ {{repo['stargazers_count']}}"
                response_lines.append(repo_line)

                if repo.get('description'):
                    response_lines.append(f"   {{repo['description']}}")
                response_lines.append("")  # Empty line for spacing

            return "\\n".join(response_lines)

    except httpx.HTTPStatusError as e:
        return f"GitHub API error: {{e.response.status_code}} - {{e.response.text}}"
    except Exception as e:
        return f"Error fetching GitHub repositories: {{str(e)}}"


class StreamingQueue:
    """Simple async queue for streaming responses."""

    def __init__(self):
        self._queue = asyncio.Queue()
        self._finished = False

    async def put(self, item: str) -> None:
        """Add an item to the queue."""
        await self._queue.put(item)

    async def finish(self) -> None:
        """Mark the queue as finished and add sentinel value."""
        self._finished = True
        await self._queue.put(None)

    async def stream(self):
        """Stream items from the queue until finished."""
        while True:
            item = await self._queue.get()
            if item is None and self._finished:
                break
            yield item


# Initialize streaming queue
queue = StreamingQueue()


async def on_auth_url(url: str) -> None:
    """Callback for authentication URL."""
    print(f"Authorization URL: {{url}}")
    await queue.put(f"Authorization URL: {{url}}")


def extract_response_text(response) -> str:
    """Extract text content from agent response."""
    if isinstance(response.message, dict):
        content = response.message.get('content', [])
        if isinstance(content, list):
            return "".join(
                item.get('text', '') for item in content
                if isinstance(item, dict) and 'text' in item
            )
    return str(response.message)


def needs_authentication(response_text: str) -> bool:
    """Check if response indicates authentication is required."""
    auth_keywords = [
        "authentication", "authorize", "authorization", "auth",
        "sign in", "login", "access", "permission", "credential",
        "need authentication", "requires authentication"
    ]
    return any(keyword.lower() in response_text.lower() for keyword in auth_keywords)


async def agent_task(user_message: str) -> None:
    """Execute agent task with authentication handling."""
    global github_access_token

    try:
        await queue.put("Begin agent execution")

        # Initial agent call
        response = agent(user_message)
        response_text = extract_response_text(response)

        # Check if authentication is needed
        if needs_authentication(response_text):
            await queue.put("Authentication required for GitHub access. Starting authorization flow...")

            try:
                github_access_token = await need_token_3LO_async(access_token='')
                await queue.put("Authentication successful! Retrying GitHub request...")

                # Retry with authentication
                response = agent(user_message)
            except Exception as auth_error:
                print(f"Authentication error: {{auth_error}}")
                await queue.put(f"Authentication failed: {{str(auth_error)}}")
                return

        await queue.put(response.message)
        await queue.put("End agent execution")

    except Exception as e:
        await queue.put(f"Error: {{str(e)}}")
    finally:
        await queue.finish()


@requires_access_token(
    provider_name="github-provider",
    scopes=["repo", "read:user"],
    auth_flow='USER_FEDERATION',
    on_auth_url=on_auth_url,
    force_authentication=False,  # ← Changed to False - will use cached token!
    callback_url="{oauth2_callback_url_for_agent}"  # ← Callback URL determined from notebook environment
)
async def need_token_3LO_async(*, access_token: str) -> str:
    """Handle 3LO authentication flow."""
    global github_access_token
    github_access_token = access_token
    return access_token


# Create agent instance
agent = Agent(
    model="global.anthropic.claude-haiku-4-5-20251001-v1:0",
    tools=[inspect_github_repos],
    system_prompt="""You are a GitHub assistant. Use the inspect_github_repos tool to fetch private repositories data.
    The inspect_github_repos tool handles token exchange and proper authentication with the GitHub API 
    to obtain private information for the user."""
)


@app.entrypoint
async def agent_invocation(payload):
    """Main entrypoint for agent invocation."""
    user_message = payload.get(
        "prompt",
        "No prompt found in input, please guide customer to create a JSON payload with prompt key"
    )

    # Create and start the agent task
    task = asyncio.create_task(agent_task(user_message))

    async def stream_with_task():
        """Stream results while ensuring task completion."""
        async for item in queue.stream():
            yield item
        await task  # Ensure task completes

    return stream_with_task()


if __name__ == "__main__":
    app.run()
'''

# Write the file
with open("github_agent.py", "w") as f:
    f.write(github_agent_code)

print("✅ github_agent.py written successfully with embedded callback URL")

## AgentCore Runtime へのエージェントのデプロイ
CreateAgentRuntime 操作は包括的な構成オプションをサポートしており、コンテナイメージ、環境変数、暗号化設定を指定できます。また、クライアントがエージェントと通信する方法を制御するためのプロトコル設定（HTTP、MCP）と認可メカニズムも構成できます。

注意：運用のベストプラクティスは、コードをコンテナとしてパッケージ化し、CI/CD パイプラインと IaC を使用して ECR にプッシュすることです

このチュートリアルでは、Amazon Bedrock AgentCode Python SDK を使用して、アーティファクトを簡単にパッケージ化し、AgentCore Runtime にデプロイできます。

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session

boto_session = Session()
region = boto_session.region_name
print(f"Region: {region}")

discovery_url = cognito_config.get("discovery_url")
client_id = cognito_config.get("client_id")

agentcore_runtime = Runtime()

response = agentcore_runtime.configure(
    entrypoint="github_agent.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name="strands_agent_github",
    authorizer_configuration={
        "customJWTAuthorizer": {
            "discoveryUrl": discovery_url,
            "allowedClients": [client_id]
        }
    }
)
response

## AgentCore 構成の確認

In [ ]:
!cat .bedrock_agentcore.yaml

### AgentCore Runtime へのエージェントの起動

Docker ファイルが準備できたので、エージェントを AgentCore Runtime に起動しましょう。これにより、Amazon ECR リポジトリと AgentCore Runtime が作成されます

<div style="text-align:left">
    <img src="images/launch.png" width="75%"/>
</div>

In [ ]:
from oauth2_callback_server import get_oauth2_callback_url

# Deploy the agent to AgentCore Runtime and get deployment details
launch_result = agentcore_runtime.launch()
print(launch_result)

if launch_result.agent_id:
    # Extract the workload name from the deployed agent's ID for identity management
    workload_name = launch_result.agent_id
    # Retrieve the current workload identity configuration from AgentCore Identity
    workload_identity = identity_client.get_workload_identity(name=workload_name)
    # Extract existing OAuth2 callback URLs that are already registered for this workload
    allowed_resource_oauth_2_return_urls = workload_identity.get("allowedResourceOauth2ReturnUrls") or []
    # Get the local OAuth2 callback server URL for session binding (localhost:9090/oauth2/callback)
    oauth2_callback_url = get_oauth2_callback_url()
    print(f"Updating workload {workload_name} with callback url {oauth2_callback_url}")

    # Register the local callback URL with the workload identity to enable OAuth2 session binding
    updated_workload_identity = identity_client.update_workload_identity(
        name=workload_name,
        allowed_resource_oauth_2_return_urls=[*allowed_resource_oauth_2_return_urls, oauth2_callback_url],
    )
    print(updated_workload_identity)

#### 自動作成されたロールに追加の必要なポリシーを追加

これをモデルにアクセスしたことがない新しいアカウントで使用している場合、エージェントがモデルにアクセスできるように、自動作成されたロールに追加の必要なポリシーを追加する必要があります。

In [ ]:
import json
import boto3
agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)

runtime_response = agentcore_control_client.get_agent_runtime(
    agentRuntimeId=launch_result.agent_id
)
runtime_role = runtime_response['roleArn']

policies_to_add = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "BedrockModelAccess",
            "Effect": "Allow",
            "Action": [
                "aws-marketplace:ViewSubscriptions",
                "aws-marketplace:Subscribe"
            ],
            "Resource": "*"
        }
    ]
}
iam_client = boto3.client(
    'iam',
    region_name=region
)

response = iam_client.put_role_policy(
    PolicyDocument=json.dumps(policies_to_add),
    PolicyName="outbound_policies",
    RoleName=runtime_role.split("/")[1],
)

### AgentCore Runtime のステータス確認
AgentCore Runtime をデプロイしたので、デプロイステータスを確認しましょう

In [ ]:
import time

status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
print(f"Final status: {status}")

### AgentCore Runtime の呼び出し

最後に、ペイロードを使用して AgentCore Runtime を呼び出すことができます

エージェントが「inspect_github_repos」ツールを呼び出し、3 Legged OAuth フローをトリガーすることに気づくでしょう。認可 URL が表示されます。認可 URL をクリックするか、新しいブラウザセッション/タブにコピー＆ペーストしてユーザー同意フローを完了します。
認可が完了すると、資格情報プロバイダー「github-provider」が GitHub からアクセストークンをフェッチし、ツールの実行を完了して GitHub アカウントからプライベートリポジトリをフェッチします。

<div style="text-align:left">
    <img src="images/invoke.png" width=75%"/>
</div>

In [ ]:
import subprocess
from oauth2_callback_server import store_token_in_oauth2_callback_server, wait_for_oauth2_server_to_be_ready

bearer_token = reauthenticate_user(cognito_config.get("client_id"))

oauth2_callback_server_cmd = [sys.executable, "oauth2_callback_server.py", "--region", region]
oauth2_callback_server_process = subprocess.Popen(oauth2_callback_server_cmd)

try:
    # Start the OAuth2 callback server
    successfully_started_oauth2_server = wait_for_oauth2_server_to_be_ready()
    if not successfully_started_oauth2_server:
        print("Failed to start OAuth2 callback server to handle session binding "
              "(https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/oauth2-authorization-url-session-binding.html)")
    else:
        store_token_in_oauth2_callback_server(bearer_token)
        invoke_response = agentcore_runtime.invoke(
            {"prompt": "What are my private repositories?"},
            bearer_token=bearer_token
        )
        print(invoke_response)
finally:
    oauth2_callback_server_process.terminate()

## オプション - Streamlit アプリを使用したエージェントのテスト

素敵なチャットインターフェースを提供する Streamlit Web アプリケーションを使用して、デプロイされたエージェントをテストできます。このディレクトリの `chatbot_app_cognito.py` ファイルは、以下の機能を持つ Web ベースのチャットボットを作成します：

- `.bedrock_agentcore.yaml` から構成を自動的に読み取る
- Cognito 認証を提供
- ストリーミングレスポンス付きのモダンなチャットインターフェースを表示
- GitHub アクセスのための 3LO OAuth フローを処理

### Streamlit アプリの実行

Streamlit アプリはいくつかの方法で実行できます：

#### オプション 1：Jupyter Notebook（現在のディレクトリ）から実行
- 以下のセルを実行して、このノートブックから直接 Streamlit アプリを開始します：
- ログイン：資格情報 testuser / MyPassword123!（Cognito セットアップで作成されたデフォルトのテストユーザー）を使用
- 「Tell me a joke」のような簡単なプロンプトをテスト
- 「What are my private repositories?」のような `inspect_github_repos` ツールをトリガーするプロンプトでテスト
- 認可 URL が返されます。URL をクリックするか、新しいブラウザタブ/ウィンドウに URL をコピー＆ペーストしてユーザー同意フローを完了します。

In [ ]:
import subprocess
from chatbot_app_cognito import get_streamlit_url

# Change to the current directory where the chatbot_app_cognito.py file is located
notebook_dir = os.getcwd()

# Start the Streamlit app
print("Starting Streamlit app...")

oauth2_callback_server_cmd = [sys.executable, "oauth2_callback_server.py", "--region", region]
oauth2_callback_server_process = subprocess.Popen(oauth2_callback_server_cmd)

try:
    wait_for_oauth2_server_to_be_ready()

    # Run streamlit in the current directory
    process = subprocess.Popen([
        sys.executable, "-m", "streamlit", "run", "chatbot_app_cognito.py",
        "--server.port=8501", "--server.showEmailPrompt=false"
    ],
        cwd=notebook_dir,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True
    )

    # Print the output as it comes
    for line in iter(process.stdout.readline, ''):
        if line:
            if "8501" in line:
                print("\n🎉 Streamlit app is ready!")
                streamlit_url = get_streamlit_url()
                print(f'\n🚀 Streamlit Application URL:\n{streamlit_url}\n')
                print("⚠️ To stop the app, interrupt the kernel or press Ctrl+C in the terminal")
                break

except KeyboardInterrupt:
    print("\nStreamlit app stopped.")
    oauth2_callback_server_process.terminate()
    process.terminate()
except Exception as e:
    print(f"Error starting Streamlit app: {e}")

## クリーンアップ（オプション）

作成した AgentCore Runtime をクリーンアップしましょう

In [ ]:
# agentcore_control_client = boto3.client(
#     'bedrock-agentcore-control',
#     region_name=region
# )
# ecr_client = boto3.client(
#     'ecr',
#     region_name=region
    
# )

# iam_client = boto3.client('iam')

# runtime_delete_response = agentcore_control_client.delete_agent_runtime(
#     agentRuntimeId=launch_result.agent_id
# )

# response = ecr_client.delete_repository(
#     repositoryName=launch_result.ecr_uri.split('/')[1],
#     force=True
# )

# policies = iam_client.list_role_policies(
#     RoleName=agentcore_iam_role['Role']['RoleName'],
#     MaxItems=100
# )

# for policy_name in policies['PolicyNames']:
#     iam_client.delete_role_policy(
#         RoleName=agentcore_iam_role['Role']['RoleName'],
#         PolicyName=policy_name
#     )
# iam_response = iam_client.delete_role(
#     RoleName=agentcore_role_name
# )